<center><font size=6> 选择合适的交易币种 </font></center>
目前还没完成

1. 获取全部可交易的货币对，quote_currency 应该是 USDT
2. 选择其中价格较低的部分(100内)
3. 计算价格回归的斜率，排除斜率较大的（封装为函数）
4. 计算方差，选择方差较大的（封装为函数）

In [1]:
from huobi.client.market import MarketClient
from huobi.constant import *
from huobi.utils import *

In [2]:
import pandas as pd

In [3]:
def get_market_prices(symbol:str, interval, size:int):
    '''获取市场数据，以Dataframe形式返回'''
    market_client = MarketClient(init_log=True)
    list_obj = market_client.get_candlestick(symbol, interval, size)
    # LogInfo.output("---- {interval} candlestick for {symbol} ----".format(interval=interval, symbol=symbol))
    # LogInfo.output_list(list_obj)
    price_df = Candle_2_Dataframe(list_obj)  # 注意时间顺序，第一条记录是最新记录
    price_df = price_df[::-1].reset_index(drop=True)  
    return price_df


def Candle_2_Dataframe(list_obj):
    '''把奇怪的huobi格式转化为Dataframe'''
    df = pd.DataFrame({
        'open':list(map(lambda x: x.open, list_obj)),
        'close': list(map(lambda x: x.close, list_obj)),
        'high': list(map(lambda x: x.high, list_obj)),
        'low': list(map(lambda x: x.low, list_obj)),
        'vol': list(map(lambda x: x.vol, list_obj)),
        'count': list(map(lambda x: x.count, list_obj)),
        'amount': list(map(lambda x: x.amount, list_obj)),
    })
    return df

## 1. 获取全部可交易的货币对
quote_currency 应该是 USDT

In [13]:
generic_client = GenericClient()
list_obj = generic_client.get_exchange_currencies()

In [17]:
len(list_obj)

894

In [ ]:
from huobi.client.generic import GenericClient
from huobi.utils import *


In [25]:
# 获取全部的交易币种
generic_client = GenericClient()
currency_list_obj = generic_client.get_exchange_symbols()
trade_currency_list = []
if len(list_obj):
    for idx, currency_info in enumerate(currency_list_obj):
        if currency_info.quote_currency == 'usdt' and currency_info.state == 'online' :
            trade_currency_list.append(currency_info)

In [26]:
currency_list_obj[0].print_object()

Base Currency : ant
Quote Currency : eth
Price Precision : 6
Amount Precision : 2
Symbol Partition : main
Symbol : anteth
State : offline
Value Precision : 8
Min Order Amount : 0.01
Max Order Amount : 250000
Min Order Value : 0.001
Leverage Ratio : 0
Minimum order amount (Limit Order) : 0.01
Max order amount (Limit Order) : 250000
Min order amount (Sell Market Order) : 0.01
Max order amount (Sell Market Order) : 25000
Max order value (Buy Market Order) : 380
Max order value (In USDT) : 0


In [28]:
len(trade_currency_list)

599

## 2. 选择其中价格较低的部分(100内)

In [29]:
symbol_list = [x.symbol for x in trade_currency_list]

In [30]:
market_client = MarketClient()

In [34]:
symbols_low_price = []
for symbol in symbol_list:
    trade_info = None
    try:
        trade_info = market_client.get_market_trade(symbol=symbol)
    except:
        print('{0}获取价格信息失败'.format(symbol))
        continue
    if len(trade_info):
        if trade_info[0].price < 100:
            symbols_low_price.append(symbol)
    else:
        print('没有获取到价格信息')

In [35]:
len(symbols_low_price)

585

In [36]:
import numpy as np

In [38]:
symbols_low_price = np.array(symbols_low_price)

In [39]:
np.save('symbols_low_price.npy',symbols_low_price)

## 3. 计算价格回归的斜率
排除斜率较大的（封装为函数）

### 3.1 从交易量角度判断流动性

In [ ]:
market_client = MarketClient()

In [67]:
volume_list = []
for symbol in symbols_low_price:
    obj = market_client.get_market_detail(symbol)
    volume_list.append(obj.vol)

In [68]:
import pandas as pd

In [70]:
Currency_info_df = pd.DataFrame({
    'symbol':symbols_low_price,
    'volume':volume_list,})


In [97]:
symbols_candidates = Currency_info_df.sort_values(by='volume', ascending=False)[:50].symbol.values
Currency_candidate_df = Currency_info_df.sort_values(by='volume', ascending=False)[:50]
Currency_candidate_df.reset_index(inplace=True, drop=True)

### 3.2 计算过去24h的分钟频率方差

In [105]:
interval = CandlestickInterval.MIN1
std_list = []
for symbol in symbols_candidates:
    price_df = get_market_prices(symbol, interval, size=2000)
    std_list.append(np.std(price_df.close))

In [107]:
Currency_candidate_df['std'] = std_list

In [116]:
# 选择方差较大的20个symbols
Currency_candidate_df = Currency_candidate_df.sort_values(by='std', ascending=False)[:20]
candidate_symbols = Currency_candidate_df.symbol.values

In [120]:
# 保存起来
np.save('candidate_symbols', candidate_symbols)

In [121]:
candidate_symbols

array(['compusdt', 'aaveusdt', 'solusdt', 'bsvusdt', 'aptusdt', 'etcusdt',
       'ordiusdt', 'avaxusdt', 'fxsusdt', 'axsusdt', 'maskusdt',
       'tonusdt', 'filusdt', 'dydxusdt', 'revousdt', 'arbusdt', 'ldousdt',
       'yggusdt', 'xrpusdt', 'maticusdt'], dtype=object)

In [125]:
base =  [x[:-4] for x in candidate_symbols]

In [128]:
 'comp' in base

True

In [143]:
currency_info_df = pd.DataFrame()

currency_info_df["symbol"] = candidate_symbols
currency_info_df["base_currency"] = base
currency_info_df = currency_info_df.set_index("symbol")

In [149]:
currency_info_df['x'] = 0

In [151]:
currency_info_df.loc["aaveusdt"]['x'] = 10

F:\ProgramData\Anaconda3\envs\pythonlab\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [158]:
currency_info_df.loc["aaveusdt",'x'] = 10

In [160]:
currency_info_df.index[0]

'compusdt'

In [167]:
x = [10]

In [168]:
a = [x]

In [169]:
a

[[10]]

In [170]:
x = [3]

In [172]:
if 0:
    print('xxx')